In [1]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
import boto3
from nba_api.stats.endpoints import LeagueGameFinder, LeagueDashPlayerBioStats
import psycopg2


pd.set_option('display.max_columns', 500)

# Example usage
db_name = "thebigone"
username = "ajwin"
password = "CharlesBark!23"
host = "nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com"


In [ ]:
from nba_api.stats.endpoints import leaguegamefinder

seasons = []
for i in list(range(1983,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]


In [3]:
# CREATE DATABASE
# import boto3

# # Create RDS Client
# rds_client = boto3.client('rds')

# def create_rds_instance(db_instance_identifier, db_name, master_username, master_user_password):
#     try:
#         response = rds_client.create_db_instance(
#             DBInstanceIdentifier=db_instance_identifier,
#             AllocatedStorage=20,  # Adjust as needed
#             DBName=db_name,
#             Engine='postgres',  # Using PostgreSQL, can switch if needed
#             MasterUsername=master_username,
#             MasterUserPassword=master_user_password,
#             DBInstanceClass='db.t3.micro',  # Adjust based on size
#             PubliclyAccessible=False,
#             MultiAZ=False,
#             StorageType='gp2',
#             BackupRetentionPeriod=7,
#         )
#         print(f"RDS instance {db_instance_identifier} creation initiated.")
#         return response
#     except Exception as e:
#         print(f"Error creating RDS instance: {str(e)}")

# create_rds_instance("nba-rds-instance","thebigone","ajwin","CharlesBark!23")


RDS instance nba-rds-instance creation initiated.


{'DBInstance': {'DBInstanceIdentifier': 'nba-rds-instance',
  'DBInstanceClass': 'db.t3.micro',
  'Engine': 'postgres',
  'DBInstanceStatus': 'creating',
  'MasterUsername': 'ajwin',
  'DBName': 'thebigone',
  'AllocatedStorage': 20,
  'PreferredBackupWindow': '06:40-07:10',
  'BackupRetentionPeriod': 7,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-048e804df880ce6ca',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSubnetGroup': {'DBSubnetGroupName': 'default',
   'DBSubnetGroupDescription': 'default',
   'VpcId': 'vpc-0da11b2b953a7cab5',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetIdentifier': 'subnet-02fedebb493b9a06e',
     'SubnetAvailabilityZone': {'Name': 'us-east-1c'},
     'SubnetOutpost': {},
     'SubnetStatus': 'Active'},
    {'SubnetIdentifier': 'subnet-0c8f0f9dc389265c1',
     'SubnetAvailabilityZone': {'Name': 'us-east-1e'},
     'S

In [9]:
seasons = []
for i in list(range(1983,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]


def get_games_data():
    # Get game data
    gamesl = []
    for i in seasons:
        gamefinder = LeagueGameFinder(league_id_nullable='00',season_nullable=i).get_data_frames()[0]
        gamesl.append(gamefinder)
        time.sleep(1)
    gamehistory = pd.concat(gamesl,axis=0)
    gamehistory['GAME_DATE'] =  pd.to_datetime(gamehistory['GAME_DATE'])
    return gamehistory


def get_players_data():
    # Get player data
    playersl = []
    for i in seasons:
        playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
        playerfinder['season']=i
        playersl.append(playerfinder)
        time.sleep(.5)
    allplayers = pd.concat(playersl,axis=0)
    return allplayers


In [10]:

# Connect to RDS database
def connect_to_rds(db_name, username, password, host, port=5432):
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=username,
            password=password,
            host=host,
            port=port
        )
        print("Connected to RDS database")
        return conn
    except Exception as e:
        print(f"Error connecting to RDS: {str(e)}")
        return None

# Create a table in PostgreSQL
import psycopg2

# Data type mapping from pandas to PostgreSQL
def map_dtype_to_postgresql(dtype):
    if dtype == 'int64':
        return 'INTEGER'
    elif dtype == 'float64':
        return 'FLOAT'
    elif dtype == 'bool':
        return 'BOOLEAN'
    elif dtype == 'datetime64[ns]':
        return 'TIMESTAMP'
    else:
        return 'TEXT'  # Default to TEXT for object or string types

# Create a table in PostgreSQL based on the dataframe schema
def create_table(conn, table_name, dataframe):
    try:
        cursor = conn.cursor()

        # Generate the SQL statement for creating a table
        columns = []
        for col, dtype in zip(dataframe.columns, dataframe.dtypes):
            pg_type = map_dtype_to_postgresql(dtype.name)
            columns.append(f"{col} {pg_type}")
        
        # Combine all columns into a SQL CREATE TABLE statement
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(columns)});"
        
        cursor.execute(create_table_query)
        conn.commit()
        print(f"Table {table_name} created successfully with inferred schema.")
    except Exception as e:
        print(f"Error creating table: {str(e)}")


# Insert data into the table
def insert_data(conn, table_name, dataframe):
    cursor = conn.cursor()
    for _, row in dataframe.iterrows():
        placeholders = ', '.join(['%s'] * len(row))
        columns = ', '.join(dataframe.columns)
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING;"
        cursor.execute(insert_query, tuple(row))
    conn.commit()
    print(f"Data inserted into {table_name}.")


conn = connect_to_rds(db_name, username, password, host)

# Get and insert games data
games_data = get_games_data()
create_table(conn, "games", games_data)
insert_data(conn, "games", games_data)

# Get and insert players data
players_data = get_players_data()
create_table(conn, "players", players_data)
insert_data(conn, "players", players_data)


Connected to RDS database


C:\Users\ajwin\AppData\Local\Temp\ipykernel_54512\3690715273.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gamehistory = pd.concat(gamesl,axis=0)


Table games created successfully.
Data inserted into games.


C:\Users\ajwin\AppData\Local\Temp\ipykernel_54512\3690715273.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  allplayers = pd.concat(playersl,axis=0)


Table players created successfully.
Data inserted into players.


Connected to RDS database


C:\Users\ajwin\AppData\Local\Temp\ipykernel_54512\270539326.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


In [ ]:
#############################################################
#############################################################

# Query the database and return results as a pandas dataframe
def query_database_to_dataframe(conn, query):
    try:
        # Read the query result into a pandas dataframe
        dataframe = pd.read_sql(query, conn)
        return dataframe
    except Exception as e:
        print(f"Error executing query: {str(e)}")
        return None

# Connect to the RDS database
conn = connect_to_rds(db_name, username, password, host)

# Define your SQL query
query = "SELECT * FROM players;"  # Example query: fetch 10 rows from the 'games' table

# Execute the query and store the result in a pandas dataframe
if conn:
    result_df = query_database_to_dataframe(conn, query)


# Close the connection
conn.close()

In [19]:
result_df

,player_id,player_name,team_id,team_abbreviation,age,player_height,player_height_inches,player_weight,college,country,draft_year,draft_round,draft_number,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,1631260,AJ Green,1610612749.0,MIL,24.0,6-4,76.0,190,Northern Iowa,USA,Undrafted,Undrafted,Undrafted,56,252,64,30,3.5,0.015,0.089,0.15,0.617,0.068,2023-24
1,1631100,AJ Griffin,1610612737.0,ATL,20.0,6-6,78.0,220,Duke,USA,2022,1,16,20,48,18,5,-14.0,0.011,0.108,0.167,0.382,0.041,2023-24
2,1630639,AJ Lawson,1610612742.0,DAL,23.0,6-6,78.0,179,South Carolina,Canada,Undrafted,Undrafted,Undrafted,42,136,50,20,1.2,0.039,0.105,0.18,0.519,0.089,2023-24
3,203932,Aaron Gordon,1610612743.0,DEN,28.0,6-8,80.0,235,Arizona,USA,2014,1,4,73,1013,471,259,8.7,0.078,0.129,0.174,0.607,0.147,2023-24
4,1628988,Aaron Holiday,1610612745.0,HOU,27.0,6-0,72.0,185,UCLA,USA,2018,1,23,78,514,123,140,2.9,0.017,0.075,0.158,0.578,0.157,2023-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13413,335,Willie Anderson,1610612748.0,MIA,30.0,6-8,80.0,200,Georgia,USA,1988,1,10,28,83,42,34,-1.4,0.054,0.104,0.142,0.57,0.191,1996-97
13414,416,Willie Burton,1610612737.0,ATL,29.0,6-8,80.0,210,Minnesota,USA,1990,1,9,24,148,41,11,-7.0,0.03,0.082,0.207,0.507,0.056,1996-97
13415,1365,Xavier McDaniel,1610612751.0,NJN,34.0,6-7,79.0,218,Wichita State,USA,1985,1,4,62,346,318,65,0.0,0.098,0.162,0.163,0.439,0.085,1996-97
13416,212,Yinka Dare,1610612751.0,NJN,24.0,7-0,84.0,265,George Washington,USA,1994,1,14,41,57,82,3,0.5,0.097,0.163,0.118,0.406,0.013,1996-97
